In [ ]:
import matplotlib.pyplot as plt

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

%matplotlib inline

In [ ]:
import numpy as np
import networkx as nx
from collections import Counter

In [ ]:
import sys
sys.path.append('../src')
from random_nested_hypergraph import random_nested_hypergraph
from random_degree_model import random_degree_hypergraph
from encapsulation_dag import encapsulation_dag, overlap_dag, overlap_graph

In [ ]:
N = 20
max_size = 4
H = 5
epsilons = {2: 1.0, 3: 1.0, 4:1.0}
max_size_overlap = 1

In [ ]:
def get_plot_dag(N, max_size, H, epsilons, max_size_overlap=-1):
    hyperedges = random_nested_hypergraph(N, max_size, H, epsilons, max_size_overlap=max_size_overlap)
    dag, nth, he_map = encapsulation_dag(hyperedges)
    size_dict = dict()
    for node in dag.nodes():
        size_dict[node] = len(node)

    nx.set_node_attributes(dag, size_dict, name="subset")
    return dag, hyperedges

In [ ]:
def get_plot_overdag(N, max_size, H, epsilons, max_size_overlap=-1):
    hyperedges = random_nested_hypergraph(N, max_size, H, epsilons, max_size_overlap=max_size_overlap)
    dag, nth, he_map = overlap_dag(hyperedges)
    size_dict = dict()
    for node in dag.nodes():
        size_dict[node] = len(node)

    nx.set_node_attributes(dag, size_dict, name="subset")
    return dag, hyperedges

In [ ]:
def get_plot_overlap(N, max_size, H, epsilons, max_size_overlap=-1):
    hyperedges = random_nested_hypergraph(N, max_size, H, epsilons, max_size_overlap=max_size_overlap)
    dag, nth, he_map = overlap_graph(hyperedges)
    size_dict = dict()
    for node in dag.nodes():
        size_dict[node] = len(node)

    nx.set_node_attributes(dag, size_dict, name="subset")
    return dag, hyperedges

In [ ]:
# Varying both
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(12, 6), squeeze=False)

for row_idx, ep2 in enumerate([1.0, 0.5, 0.0]):
    for col_idx, ep3 in enumerate([1.0, 0.5, 0.0]):
        epsilons = {2: ep2, 3: ep3}
        dag, _ = get_plot_dag(N, max_size, H, epsilons, max_size_overlap=max_size_overlap)
        axs[row_idx][col_idx].set_axis_off()
        multipart_layout = nx.multipartite_layout(dag, align="horizontal")
        nx.draw_networkx(dag, node_size=40, arrowsize=2, pos=multipart_layout, ax=axs[row_idx][col_idx],
                         with_labels=False, node_color="dodgerblue", alpha=0.5)
        axs[row_idx][col_idx].set(title=fr"$\epsilon_2={ep2}, \epsilon_3={ep3}$")
        axs[row_idx][col_idx].title.set_fontsize(15)
        if col_idx == 0:
            # Annoying one-liner to get the y-position of each layer
            y_dict = {size:[multipart_layout[node]
                            for node in dag.nodes()
                            if len(node) == size][0][1]
                      for size in range(2, max_size+1)}
            for size in y_dict:
                axs[row_idx][col_idx].text(-1.55, y_dict[size], fr"$s={size}$", size=13)
            #if row_idx == 0:
            #    axs[row_idx][col_idx].text(-1.8, y_dict[max(y_dict.keys())]+0.05, "Hyperedge Size", size=15)
fig.subplots_adjust(wspace=0.0)
fig.suptitle(r"Encapsulation DAGs for Varying $\epsilon_s$", size=16)
fig.tight_layout()
#fig.savefig("../results/plots/nested-dag-example.pdf")

In [ ]:
# Overlap DAG
with_uniform = False
colors = {1:"#e5d262", 2:"#7de5b8", 3: "#9be95d"}
for num in range(1):
    if with_uniform:
        fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(12, 6), squeeze=False)
    else:
        fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(12, 6), squeeze=False)
    for row_idx, ep2 in enumerate([1.0, 0.5, 0.0]):
        for col_idx, ep3 in enumerate([1.0, 0.5, 0.0]):
            epsilons = {2: ep2, 3: ep3}
            dag, hyperedges = get_plot_overdag(N, max_size, H, epsilons, max_size_overlap = 1)
            multipart_layout = nx.multipartite_layout(dag, align="horizontal")
            nx.draw_networkx_nodes(dag, multipart_layout, node_size=40,
                                   node_color="dodgerblue", alpha=0.5, ax=axs[row_idx][col_idx])
            edgelist = [(u,v) for u,v,data in dag.edges(data=True) if data["weight"] < 1.0]
            edge_color = np.array([float(dag.edges[(u,v)]["weight"]) for u,v in edgelist])
            #edge_color = [colors[dag.edges[(u,v)]["overlap"]] for u,v in edgelist]
            nx.draw_networkx_edges(dag, multipart_layout, edgelist=edgelist,
                                   edge_color=edge_color, edge_cmap=plt.cm.Greens,
                                   arrowsize=2, ax=axs[row_idx][col_idx], alpha=0.5)
            edgelist = [(u,v) for u,v,data in dag.edges(data=True) if data["weight"] == 1.0]
            nx.draw_networkx_edges(dag, multipart_layout, edgelist=edgelist, edge_color="black",
                                   arrowsize=2, ax=axs[row_idx][col_idx], alpha=0.4)
            axs[row_idx][col_idx].set(title=fr"$\epsilon_2={ep2}, \epsilon_3={ep3}$")
            if col_idx == 0:
                # Annoying one-liner to get the y-position of each layer
                y_dict = {size:[multipart_layout[node]
                                for node in dag.nodes()
                                if len(node) == size][0][1]
                          for size in range(2, max_size+1)}
                for size in y_dict:
                    axs[row_idx][col_idx].text(-1.55, y_dict[size], fr"$s={size}$")

    if with_uniform:
        # Add uniform random for comparison
        size_list = [len(he) for he in hyperedges]
        size_dist = Counter(size_list)
        uniform_hyperedges = random_degree_hypergraph(N, size_dist, correlation="uncorrelated")
        dag, nth, he_map = overlap_dag(uniform_hyperedges)
        size_dict = dict()
        for node in dag.nodes():
            size_dict[node] = len(node)

        nx.set_node_attributes(dag, size_dict, name="subset")
        col_idx = 3
        row_idx = 1
        axs[row_idx][col_idx].set_axis_off()
        multipart_layout = nx.multipartite_layout(dag, align="horizontal")
        nx.draw_networkx_nodes(dag, multipart_layout, node_size=40,
                               node_color="dodgerblue", alpha=0.5, ax=axs[row_idx][col_idx])
        edgelist = [(u,v) for u,v,data in dag.edges(data=True) if data["weight"] < 1.0]
        edge_color = np.array([float(dag.edges[(u,v)]["weight"]) for u,v in edgelist])
        #edge_color = [colors[dag.edges[(u,v)]["overlap"]] for u,v in edgelist]
        nx.draw_networkx_edges(dag, multipart_layout, edgelist=edgelist,
                               edge_color=edge_color, edge_cmap=plt.cm.Greens,
                               arrowsize=2, ax=axs[row_idx][col_idx], alpha=0.5)
        edgelist = [(u,v) for u,v,data in dag.edges(data=True) if data["weight"] == 1.0]
        nx.draw_networkx_edges(dag, multipart_layout, edgelist=edgelist, edge_color="black",
                               arrowsize=2, ax=axs[row_idx][col_idx], alpha=0.4)
        axs[row_idx][col_idx].set(title=fr"Uniform")

    for row_idx in range(3):
        for col_idx in range(3):
            axs[row_idx][col_idx].set_axis_off()

    fig.subplots_adjust(wspace=0.0)
    fig.suptitle(r"Overlap Structures for Varying $\epsilon_s$", size=16)
    fig.tight_layout()
    #fig.savefig(f"../results/plots/nested-overdag-example.pdf")